# Depth Image Transformations

In [2]:
import numpy as np
import math
import cv2
import os, os.path
import sys
import time
import pandas as pd

path = 'C:/Users/jumpr_000/Desktop/ransac_obstacle_detection/saved' #set to your desired path to save images in
frames_dir = "test_frames_straight/" #the directory holding the saved depth images in .npy format

#camera information based on the Kinect v2 hardware
CameraParams = {
  "cx":254.878,
  "cy":205.395,
  "fx":365.456,
  "fy":365.456,
  "k1":0.0905474,
  "k2":-0.26819,
  "k3":0.0950862,
  "p1":0.0,
  "p2":0.0,
}

# Kinect's physical orientation in the real world.
CameraPosition = {
    "x": 0, # actual position in meters of kinect sensor relative to the viewport's center.
    "y": 0, # actual position in meters of kinect sensor relative to the viewport's center.
    "z": 0, # height in meters of actual kinect sensor from the floor.
    "roll": 0, # angle in degrees of sensor's roll (used for INU input - trig function for this is commented out by default).
    "azimuth": 0, # sensor's yaw angle in degrees.
    "elevation": -30, # sensor's pitch angle in degrees.
}

def depthMatrixToPointCloudPos(z, scale=1000):
    """
    xyz_arr = depthMatrixToPointCloudPos(depth_frame)

    Given a depth image, converts each
    point to an XYZ coordinate and returns
    an array of these points.
    The scale is a conversion factor.
    Default converts from milimeter
    depth data to meters.
    """
    C, R = np.indices(z.shape)

    R = np.subtract(R, CameraParams['cx'])
    R = np.multiply(R, z)
    R = np.divide(R, CameraParams['fx'] * scale)

    C = np.subtract(C, CameraParams['cy'])
    C = np.multiply(C, z)
    C = np.divide(C, CameraParams['fy'] * scale)

    return np.column_stack((z.ravel() / scale, R.ravel(), -C.ravel()))

def depthToPointCloudPos(x_d, y_d, z, scale=1000):
    """
    x, y, z = (row, col, depth)
    
    Given a point from a depth image
    and its position in the image,
    returns the x, y, and z coordinates
    relative to the camera location.
    The scale is a conversion factor.
    Default converts from milimeter
    depth data to meters.
    """
    x = (x_d - CameraParams['cx']) * z / CameraParams['fx']
    y = (y_d - CameraParams['cy']) * z / CameraParams['fy']

    return x / scale, y / scale, z / scale

def applyCameraOrientation(pt):
    # Kinect Sensor Orientation Compensation
    # This runs slowly in Python as it is required to be called within a loop, but it is a more intuitive example than it's vertorized alternative (Purly for example)
    # use trig to rotate a vertex around a gimbal.
    def rotatePoints(ax1, ax2, deg):
        # math to rotate vertexes around a center point on a plane.
        hyp = np.sqrt(pt[ax1] ** 2 + pt[ax2] ** 2) # Get the length of the hypotenuse of the real-world coordinate from center of rotation, this is the radius!
        d_tan = np.arctan2(pt[ax2], pt[ax1]) # Calculate the vertexes current angle (returns radians that go from -180 to 180)

        cur_angle = np.degrees(d_tan) % 360 # Convert radians to degrees and use modulo to adjust range from 0 to 360.
        new_angle = np.radians((cur_angle + deg) % 360) # The new angle (in radians) of the vertexes after being rotated by the value of deg.

        pt[ax1] = hyp * np.cos(new_angle) # Calculate the rotated coordinate for this axis.
        pt[ax2] = hyp * np.sin(new_angle) # Calculate the rotated coordinate for this axis.

    #rotatePoints(0, 2, CameraPosition['roll']) #rotate on the Y&Z plane # Disabled because most tripods don't roll. If an Inertial Nav Unit is available this could be used)
    rotatePoints(1, 2, CameraPosition['elevation']) #rotate on the X&Z plane
    rotatePoints(0, 1, CameraPosition['azimuth']) #rotate on the X&Y plane

    # Apply offsets for height and linear position of the sensor (from viewport's center)
    pt[:] += np.float_([CameraPosition['x'], CameraPosition['y'], CameraPosition['z']])
    return pt

def applyCameraMatrixOrientation(pt):
    # Kinect Sensor Orientation Compensation
    # bacically this is a vectorized version of applyCameraOrientation()
    # uses same trig to rotate a vertex around a gimbal.
    def rotatePoints(ax1, ax2, deg):
        # math to rotate vertexes around a center point on a plane.
        hyp = np.sqrt(pt[:, ax1] ** 2 + pt[:, ax2] ** 2) # Get the length of the hypotenuse of the real-world coordinate from center of rotation, this is the radius!
        d_tan = np.arctan2(pt[:, ax2], pt[:, ax1]) # Calculate the vertexes current angle (returns radians that go from -180 to 180)

        cur_angle = np.degrees(d_tan) % 360 # Convert radians to degrees and use modulo to adjust range from 0 to 360.
        new_angle = np.radians((cur_angle + deg) % 360) # The new angle (in radians) of the vertexes after being rotated by the value of deg.

        pt[:, ax1] = hyp * np.cos(new_angle) # Calculate the rotated coordinate for this axis.
        pt[:, ax2] = hyp * np.sin(new_angle) # Calculate the rotated coordinate for this axis.

    #rotatePoints(1, 2, CameraPosition['roll']) #rotate on the Y&Z plane # Disabled because most tripods don't roll. If an Inertial Nav Unit is available this could be used)
    rotatePoints(0, 2, CameraPosition['elevation']) #rotate on the X&Z plane
    rotatePoints(0, 1, CameraPosition['azimuth']) #rotate on the X&Y

    # Apply offsets for height and linear position of the sensor (from viewport's center)
    pt[:] += np.float_([CameraPosition['x'], CameraPosition['y'], CameraPosition['z']])
    return pt

# Plane Fitting

In [3]:
def plane_fit(points):
    """
    p, n = plane_fit(points)

    Given an array, points, of shape (d,...)
    representing points in d-dimensional space,
    fit an d-dimensional plane to the points.
    Return a point, p, on the plane (the point-cloud centroid),
    and the normal, n.
    """
    from numpy.linalg import svd
    points = np.reshape(points, (np.shape(points)[0], -1)) # Collapse trialing dimensions
    assert points.shape[0] <= points.shape[1], "There are only {} points in {} dimensions.".format(points.shape[1], points.shape[0])
    ctr = points.mean(axis=1)
    x = points - ctr[:,np.newaxis]
    M = np.dot(x, x.T) # Could also use np.cov(x) here.
    return ctr, svd(M)[0][:,-1]

def get_orientation(xyz_arr, n_iter):
    """
    center, plane, theta = get_orientation(xyz_arr, num_points)

    Given an array, points, of shape (d,...)
    representing points in d-dimensional space,
    fits a plane to the array n_iter times and
    returns the average of the planes centers,
    norms, and pitch degrees.
    """
    pitch = []
    planes = []
    centers = []
    for _ in range(0, n_iter):
        rand_points = []
        while len(rand_points) < 10:
            index = random.randrange(0,len(xyz_arr))
            if not xyz_arr[index].all() == np.zeros(3).all():
                rand_points.append(xyz_arr[index])
        rand_points = np.array(rand_points).T
        ctr, P = plane_fit(rand_points)
        r = math.sqrt(P[0]**2 + P[1]**2 + P[2]**2)
        theta = math.acos(P[2]/r) * 180 / math.pi
        phi = math.atan(P[1]/P[0]) * 180 / math.pi
        pitch.append(theta)
        planes.append(P)
        centers.append(ctr)
    return np.mean(centers, axis = 0), np.mean(planes, axis = 0), np.mean(pitch)
    
    
from sklearn.linear_model import RANSACRegressor
from sklearn.datasets import make_regression

In [107]:
# Inital guess of the plane
p0 = [0.506645455682, -0.185724560275, -1.43998120646, 1.37626378129]

def f_min(X,p):
    plane_xyz = p[0:3]
    #print('plane_xyz', plane_xyz)
    distance = (plane_xyz*X).sum(axis=1) + p[3]
    return distance / np.linalg.norm(plane_xyz)

def residuals(params, signal, X):
    return f_min(X, params)

from scipy.optimize import leastsq

def fit_plane_optimize(arr):
    arr = np.random.permutation(arr)
    arr = np.reshape(arr, (np.shape(arr)[0], -1))
    ctr = arr.mean(axis=1)
    sol = leastsq(residuals, p0, args=(None, arr))[0]
    plane_xyz = sol[0:3]
    r = math.sqrt(plane_xyz[0]**2 + plane_xyz[1]**2 + plane_xyz[2]**2)
    theta = math.acos(plane_xyz[2]/r) * 180 / math.pi
    phi = math.atan(plane_xyz[1]/plane_xyz[0]) * 180 / math.pi
    return ctr[0:3], plane_xyz, phi

def get_orientation(xyz_arr, num_planes, num_points):
    """
    center, plane, theta = get_orientation(xyz_arr, num_points)

    Given an array, points, of shape (d,...)
    representing points in d-dimensional space,
    fits a plane to the array n_iter times and
    returns the average of the planes centers,
    norms, and pitch degrees.
    """
    pitch = []
    planes = []
    centers = []
    for _ in range(0, num_planes):
        rand_points = np.random.permutation(xyz_arr.T[np.all(xyz_arr.T, axis=1)])[:num_points]
        #for point in rand_points:
        #    print('Point', point)
        #for point_t in rand_points.T:
        #    print('Point.T', point_t)
        #print('Random points:\n', rand_points)
        #print('Transpose:\n', rand_points.T)
        ctr, P, phi = fit_plane_optimize(rand_points)
        pitch.append(phi)
        planes.append(P)
        centers.append(ctr)
    return np.mean(centers, axis = 0), np.mean(planes, axis = 0), np.mean(pitch)
    

# Obstacle Detection

In [108]:
import numpy as np
import math
import random
import cv2
import os, os.path
import sys
import time
import pandas as pd
from pyntcloud import PyntCloud
from IPython.display import display, HTML

def get_obstacles_with_plane(depth_frame, num_planes, num_points, dist_thresh, visualize):
    obstacles = np.zeros(depth_frame.shape) #empty image that will store the locations of detected obstacles
    img = np.uint8(depth_frame) #some opencv functions require a byte image
    kernel = np.ones((3,3),np.uint8)
    _, foreground = cv2.threshold(depth_frame,0.75,1.5, cv2.THRESH_TOZERO) 
    cropped = np.zeros(depth_frame.shape, np.float32)
    cropped[depth_frame.shape[0]//3:] = foreground[depth_frame.shape[0]//3:]
    xyz_arr = depthMatrixToPointCloudPos(cropped) #convert depth data to XYZ coordinates
    #print(fit_plane_optimize(xyz_arr.T))
    start_time = time.time()
    center, plane, theta = get_orientation(xyz_arr.T, num_planes, num_points)
    print(center, plane, theta)
    #center, plane, theta = fit_plane_optimize(xyz_arr.T, num_points)
    CameraPosition['elevation'] = -theta
    center = applyCameraOrientation(center)
    plane = applyCameraOrientation(plane)
    xyz_arr = applyCameraMatrixOrientation(xyz_arr)
    plane_img = np.zeros(len(xyz_arr))
    plane_img[xyz_arr[:,2] > dist_thresh - center[2]] = 1
    if visualize:
        xyz_arr = xyz_arr[xyz_arr[:,2] > dist_thresh - center[2]]
        points = pd.DataFrame(xyz_arr, columns=['x', 'y', 'z']) #convert XYZ coordinates to a DataFrame for pyntcloud
        '''
        cloud = PyntCloud(points)

        lines = [
            {
                # X axis
                "color": "red", 
                "vertices": [[0, 0, 0], [10, 0, 0]]
            },
            {
                # Y axis
                "color": "green",
                "vertices": [[0, 0, 0], [0, 10, 0]]
            },
            {
                # Z axis
                "color": "blue",
                "vertices": [[0, 0, 0], [0, 0, 10]]
            },
            {
                #Original norm of the plane
                "color": "pink",
                "vertices": [center.tolist(), plane.tolist()]
            }
        ]
        cloud.plot(cmap="cool", polylines=lines)
        '''

    plane_img = np.uint8(np.reshape(plane_img,(424,512)) * 255) #reshape to match depth data and convert to uint8
    plane_img = np.uint8((np.ones((424,512)) * 255) - plane_img) #invert img so pixel value corresponds to NOT ground plane
    ret, plane_img = cv2.threshold(plane_img,0,255,cv2.THRESH_BINARY) #filter points that are probaly not ground plane
    plane_img = cv2.subtract(img, plane_img)
    
    #noise removal
    kernel = np.ones((3,3),np.uint8)
    #opening = cv2.morphologyEx(plane_img ,cv2.MORPH_OPEN, kernel, iterations = 6) #erosion followed by dilation
    erode = cv2.erode(plane_img, kernel, iterations=4)
    _, thresh = cv2.threshold(erode,0,255,cv2.THRESH_BINARY)
    color = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR) #BGR image to draw labels on

    #begin contour detection
    image, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    color = cv2.drawContours(color, contours, -1, (0,255,0), 1)
    for cntr in contours:
        try:
            #calculate diamter of equivalent circle
            #this measurement is only used for checking if countours fit our bounds
            area = cv2.contourArea(cntr)
            equi_diameter = np.sqrt(4*area/np.pi)
            
            x,y,obj_length,obj_height = cv2.boundingRect(cntr)

            equi_diameter = obj_length #bounding rectangle gives a better approximation of diameter
            
            #Hardcoded Diameter Range in pixels
            LOW_DIAMETER_BOUND = 20
            HIGH_DIAMETER_BOUND = 200
            
            HIGH_MM_BOUND = 400

            HIGH_DISTANCE_BOUND = 6000
            #Original tolerances were 20 and 150

            if(equi_diameter > LOW_DIAMETER_BOUND and equi_diameter < HIGH_DIAMETER_BOUND):
                mask = np.zeros_like(img) #mask will contain the fitted and adjusted ellipse of a single obstacle
                ellipse = cv2.fitEllipse(cntr)
                rect = cv2.minAreaRect(cntr)
                box = cv2.boxPoints(rect)
                box = np.int0(box)
                mask = cv2.ellipse(mask,ellipse,(255,255,255),-1) #draw the fitted ellipse
                rows = mask.shape[0]
                cols = mask.shape[1]
                M = np.float32([[1,0,0],[0,1,equi_diameter/4]]) #shift mask down to match obstacle, not edge
                mask = cv2.warpAffine(mask,M,(cols,rows))
                mask = cv2.erode(mask, kernel, iterations=3) #erode the mask to remove background points
                img_fg = cv2.bitwise_and(depth_frame,depth_frame,mask = mask) #use the mask to isolate original depth values
                img_fg = cv2.medianBlur(img_fg,5) #median blur to further remove noise
                obstacles = cv2.add(np.float32(img_fg), np.float32(obstacles)) 
                
                mean_val = np.median(img_fg[img_fg.nonzero()]) #compute the non-zero average of obstacle depth values
                min_val, distance_to_object, min_loc, max_loc = cv2.minMaxLoc(img_fg)
                mm_diameter = (equi_diameter) * (1.0 / CameraParams['fx']) * mean_val #convert pixel diameter to mm


                moment = cv2.moments(cntr) #get the centroid of the obstacle using its moment
                cx = int(moment['m10']/moment['m00'])
                cy = int(moment['m01']/moment['m00'])

                if mean_val < HIGH_DISTANCE_BOUND and mm_diameter < HIGH_MM_BOUND: #kinect loses accuracy beyond 4.5m
                    coords = depthToPointCloudPos(cx, cy, mean_val) #convert obstacle depth to XYZ coordinate
                    #begin visualization
                    cv2.drawContours(color,[box],0,(0,0,255),1)
                    cv2.rectangle(color,(x,y),(x+obj_length,y+obj_height),(0,255,0),2)
                    font = cv2.FONT_HERSHEY_SIMPLEX
                    cv2.putText(color, "x" + str(coords[0]), (cx,cy+30), font, 0.4, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.putText(color, "y" + str(coords[1]), (cx,cy+45), font, 0.4, (0, 255, 0), 1, cv2.LINE_AA)
                    cv2.putText(color, "z" + str(mean_val), (cx,cy+60), font, 0.4, (255, 0, 0), 1, cv2.LINE_AA)
                    cv2.putText(color,"diameter = " + str(mm_diameter), (cx,cy + 15), font, 0.4, (255, 0, 0), 1, cv2.LINE_AA)

        except:
            print ("Failed to process image")
            print (sys.exc_info()[0])
            
    elapsed_time = time.time() - start_time
    print("Frame took " + str(elapsed_time) + " seconds to process")

    cv2.imshow("plane",plane_img)
    cv2.imshow("proc", thresh)
    cv2.imshow("img",depth_frame/4500.0)
    cv2.imshow("final",color)
    cv2.imshow("denoised",cropped/4500.)
    cv2.imshow("obstacles",obstacles)
    key = cv2.waitKey(delay=1)
    if key == ord('q'):
        cv2.destroyAllWindows()
        sys.exit(0)
    return color



# Single Image Testing

In [113]:
img = np.load('test_frames_arc/429.npy')
output = get_obstacles_with_plane(img, num_planes = 10, 
                                  num_points = 2000, 
                                  dist_thresh = 0.5, 
                                  visualize = True)

[0.23613381 0.34962376 0.32819382] [ 117.61233221  -56.71845128 -978.51089569] -25.899474773594147
Frame took 0.3787953853607178 seconds to process


C:\Users\jumpr_000\AppData\Roaming\Python\Python35\site-packages\numpy\core\fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\jumpr_000\AppData\Roaming\Python\Python35\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\jumpr_000\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:122: RuntimeWarning: invalid value encountered in less


# Multiple Frame Testing

In [46]:
frames_dir = 'test_frames_arc/'
directory = os.fsencode(frames_dir)
frame_i = 0
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".npy"):
        file_path = frames_dir + filename
        print(file_path)
        img = np.load(file_path)
        try:
            output = get_obstacles_with_plane(img, num_planes =20, 
                                  num_points = 2000, 
                                  dist_thresh = 0.4, 
                                  visualize = False)
            cv2.imwrite(os.path.join(path ,filename[:-4] + '.png'), output)
            frame_i += 1
        except:
            print("Could not find obstacles")
cv2.destroyAllWindows()

C:\Users\jumpr_000\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: The Windows bytes API has been deprecated, use Unicode filenames instead
  after removing the cwd from sys.path.


test_frames_arc/0.npy
[0.16577302 0.39400549 0.040437  ] [ 1.25333742 -0.2131437  -1.04019372] -9.650980053452749
Frame took 0.694582462310791 seconds to process
test_frames_arc/1.npy
[ 0.40588447  0.31414494 -0.05309001] [ 1.27324816 -0.22612183 -1.04712633] -10.070070228136302
Frame took 0.6845860481262207 seconds to process
test_frames_arc/10.npy
[0.39639979 0.46817689 0.07135641] [ 1.25855144 -0.22104757 -1.03750387] -9.964110188943291
Frame took 0.6596202850341797 seconds to process
test_frames_arc/100.npy
[0.26136401 0.21784216 0.34042689] [ 1.26057152 -0.2273556  -1.03321593] -10.223039515536607
Frame took 0.6426234245300293 seconds to process
test_frames_arc/101.npy
[0.29599156 0.42268379 0.20340776] [ 1.26897005 -0.22398157 -1.04498848] -10.010805425861378
Frame took 0.5896430015563965 seconds to process
test_frames_arc/102.npy
[0.44650392 0.45654237 0.30141895] [ 1.26373737 -0.22652404 -1.03721333] -10.162414077146549
Frame took 0.6046338081359863 seconds to process
test_fram

C:\Users\jumpr_000\AppData\Roaming\Python\Python35\site-packages\numpy\core\fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\jumpr_000\AppData\Roaming\Python\Python35\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\jumpr_000\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:122: RuntimeWarning: invalid value encountered in less


[-0.07603733  0.36755745  0.24883835] [ 1.249335   -0.22257302 -1.02676198] -10.098158754716204
Frame took 0.5836446285247803 seconds to process
test_frames_arc/106.npy
[0.38503496 0.37295188 0.2200439 ] [ 1.25469616 -0.22802282 -1.02667335] -10.30092124893373
Frame took 0.5856804847717285 seconds to process
test_frames_arc/107.npy
[0.17833082 0.33461069 0.43301565] [ 1.25249787 -0.22365697 -1.02884089] -10.129969163624459
Frame took 0.5986371040344238 seconds to process
test_frames_arc/108.npy
[0.21308279 0.26027139 0.27008108] [ 1.27057991 -0.22785786 -1.04272205] -10.166607384682333
Frame took 0.5866758823394775 seconds to process
test_frames_arc/109.npy
[0.36136388 0.56416425 0.24466657] [ 1.2631245  -0.22985519 -1.03326931] -10.312794350296608
Frame took 0.5876622200012207 seconds to process
test_frames_arc/11.npy
[ 0.0473005   0.22284273 -0.143786  ] [ 1.26660685 -0.22501653 -1.04159032] -10.069774873729331
Frame took 0.6636345386505127 seconds to process
test_frames_arc/110.npy


[0.22269816 0.3806419  0.35283609] [ 1.26289065 -0.22897995 -1.0339107 ] -10.274184052407652
Frame took 0.5696852207183838 seconds to process
test_frames_arc/151.npy
[0.42863421 0.3175657  0.26366294] [ 1.26620076 -0.23262167 -1.0335791 ] -10.408880852298777
Frame took 0.5686686038970947 seconds to process
test_frames_arc/152.npy
[0.11653508 0.28643176 0.14842609] [ 1.26657697 -0.22724146 -1.03933552] -10.170044191747095
Frame took 0.5696728229522705 seconds to process
test_frames_arc/153.npy
[0.03419374 0.53237547 0.26727651] [ 1.2627522  -0.22784929 -1.03490291] -10.228677297348232
Frame took 0.5876612663269043 seconds to process
test_frames_arc/154.npy
[0.20142671 0.20669279 0.45928586] [ 1.25743209 -0.22644884 -1.03098326] -10.206373399929731
Frame took 0.5736594200134277 seconds to process
test_frames_arc/155.npy
[0.27816817 0.32926478 0.13026366] [ 1.26514832 -0.22903569 -1.03611263] -10.26539181499707
Frame took 0.5616757869720459 seconds to process
test_frames_arc/156.npy
[0.12

...

[0.29877225 0.13807456 0.37679096] [ 1.28419671 -0.24132214 -1.04287457] -10.641670015395626
Frame took 0.46671605110168457 seconds to process
test_frames_arc/331.npy
[0.41757259 0.28296475 0.09419224] [ 1.27203489 -0.24199338 -1.0300415 ] -10.772289667888648
Frame took 0.46074700355529785 seconds to process
test_frames_arc/332.npy
[0.2666438  0.43359072 0.29210474] [ 1.26896444 -0.24538028 -1.02358416] -10.94410030638963
Frame took 0.46673154830932617 seconds to process
test_frames_arc/333.npy
[0.20250726 0.34772913 0.40504766] [ 1.27549625 -0.24501444 -1.03048181] -10.876129647196516
Frame took 0.4637324810028076 seconds to process
test_frames_arc/334.npy
[0.16407512 0.37950798 0.34211724] [ 1.26996212 -0.23889761 -1.03106451] -10.648366031563183
Frame took 0.46473217010498047 seconds to process
test_frames_arc/335.npy
[0.17844814 0.20037275 0.09676395] [ 1.2668165  -0.24421526 -1.02260124] -10.910631903364589
Frame took 0.4677298069000244 seconds to process
test_frames_arc/336.npy
[

[0.30124463 0.08112256 0.34037274] [ 1.26309041 -0.22300001 -1.0400904 ] -10.012800604517201
Frame took 0.5606954097747803 seconds to process
test_frames_arc/377.npy
[0.22652471 0.35275055 0.39936438] [ 1.26496854 -0.21916358 -1.04580496] -9.829221521455317
Frame took 0.5566725730895996 seconds to process
test_frames_arc/378.npy
[0.36552049 0.59323371 0.21444653] [ 1.25103962 -0.21804627 -1.03299335] -9.88464203831773
Frame took 0.5526633262634277 seconds to process
test_frames_arc/379.npy
[0.21788035 0.2311454  0.41518225] [ 1.26398453 -0.21884227 -1.04514226] -9.821485249235874
Frame took 0.5776674747467041 seconds to process
test_frames_arc/38.npy
[0.271316   0.23082966 0.48559433] [ 1.25777063 -0.2191626  -1.03860802] -9.883114406135926
Frame took 0.6856002807617188 seconds to process
test_frames_arc/380.npy
[0.4196757  0.33676932 0.23494938] [ 1.23211478 -0.20659884 -1.02551594] -9.522641759750641
Frame took 0.5596716403961182 seconds to process
test_frames_arc/381.npy
[0.13533729

[0.21582047 0.09655866 0.2529478 ] [ 1.25793504 -0.21764844 -1.0402866 ] -9.814854342412815
Frame took 0.5936636924743652 seconds to process
test_frames_arc/421.npy
[0.17401628 0.3920772  0.26257985] [ 1.2754497  -0.21987676 -1.05557294] -9.779832674017385
Frame took 0.5746688842773438 seconds to process
test_frames_arc/422.npy
[0.10025596 0.47428724 0.17018693] [ 1.26848271 -0.22168492 -1.0467978 ] -9.909795241381804
Frame took 0.5806658267974854 seconds to process
test_frames_arc/423.npy
[0.15532805 0.34230712 0.16444226] [ 1.26296486 -0.21599514 -1.04696972] -9.70486547605752
Frame took 0.5606577396392822 seconds to process
test_frames_arc/424.npy
[0.24324335 0.22272401 0.28636807] [ 1.26917616 -0.21926159 -1.04991456] -9.798762683504734
Frame took 0.5676531791687012 seconds to process
test_frames_arc/425.npy
[0.46755005 0.29696513 0.60541495] [ 1.27443385 -0.22146551 -1.05296835] -9.856358820746204
Frame took 0.583648681640625 seconds to process
test_frames_arc/426.npy
[0.04136342 

[0.409738   0.16989791 0.17521442] [ 1.25437981 -0.20717422 -1.04720558] -9.380833995197985
Frame took 0.600653886795044 seconds to process
test_frames_arc/467.npy
[0.28209392 0.72662759 0.33990664] [ 1.25695199 -0.21495008 -1.04200191] -9.706687862331933
Frame took 0.594651460647583 seconds to process
test_frames_arc/468.npy
[0.05804778 0.16553511 0.37211486] [ 1.27365845 -0.21153829 -1.06212016] -9.428320832682223
Frame took 0.5796661376953125 seconds to process
test_frames_arc/469.npy
[0.26958851 0.24019782 0.42983624] [ 1.25636414 -0.21184688 -1.04451726] -9.569842853791876
Frame took 0.5906543731689453 seconds to process
test_frames_arc/47.npy
[0.38584552 0.02591161 0.41200607] [ 1.26379234 -0.22305827 -1.04073407] -10.008252809202371
Frame took 0.7065742015838623 seconds to process
test_frames_arc/470.npy
[0.04878684 0.43496299 0.11762003] [ 1.26558361 -0.21217947 -1.05340414] -9.519511826035812
Frame took 0.5726706981658936 seconds to process
test_frames_arc/471.npy
[0.32385136 

[0.1876528  0.10764335 0.2683985 ] [ 1.26555788 -0.22325591 -1.04230197] -10.007637366717228
Frame took 0.672626256942749 seconds to process
test_frames_arc/62.npy
[0.2311839  0.51550978 0.53577117] [ 1.27426082 -0.22110913 -1.05315169] -9.840255388713228
Frame took 0.6675891876220703 seconds to process
test_frames_arc/63.npy
[0.11270108 0.3740893  0.08184812] [ 1.24655176 -0.21764723 -1.02890453] -9.900441433650837
Frame took 0.6596024036407471 seconds to process
test_frames_arc/64.npy
[0.42163449 0.25453291 0.7012798 ] [ 1.27860156 -0.22302202 -1.05557954] -9.893562156334436
Frame took 0.684600830078125 seconds to process
test_frames_arc/65.npy
[0.56410855 0.19111553 0.25129933] [ 1.26067028 -0.21551036 -1.04515993] -9.701534321209044
Frame took 0.6855998039245605 seconds to process
test_frames_arc/66.npy
[0.00771576 0.22491483 0.29209639] [ 1.26855121 -0.22104395 -1.04750725] -9.887691536004906
Frame took 0.6846168041229248 seconds to process
test_frames_arc/67.npy
[0.18860864 0.251